In [21]:
import pandas as pd 
import numpy as np

In [22]:
crime_sort = pd.read_hdf('../data/crime.h5', 'crime') \
.set_index('REPORTED_DATE') \
.sort_index()

In [23]:
crime_sort.head()

,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,GEO_LON,GEO_LAT,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC
REPORTED_DATE,,,,,,,
2012-01-02 00:06:00,aggravated-assault,aggravated-assault,-104.816860,39.796717,montbello,1,0
2012-01-02 00:06:00,violation-of-restraining-order,all-other-crimes,-104.816860,39.796717,montbello,1,0
2012-01-02 00:16:00,traffic-accident-dui-duid,traffic-accident,-104.971851,39.736874,cheesman-park,0,1
2012-01-02 00:47:00,traffic-accident,traffic-accident,-104.951824,39.740168,congress-park,0,1
2012-01-02 01:35:00,aggravated-assault,aggravated-assault,-104.998131,39.749922,union-station,1,0


In [24]:
crime_sort.index.max()

Timestamp('2017-09-29 06:16:00')

In [25]:
crime_sort = crime_sort[:'2017-8']

In [26]:
crime_sort.index.max()

Timestamp('2017-08-31 23:52:00')

In [27]:
all_data = crime_sort.groupby([pd.Grouper(freq='M'),
 'OFFENSE_CATEGORY_ID']).size()

In [28]:
all_data.head()

REPORTED_DATE  OFFENSE_CATEGORY_ID
2012-01-31     aggravated-assault     113
               all-other-crimes       124
               arson                    5
               auto-theft             275
               burglary               343
dtype: int64

In [29]:
all_data = all_data.sort_values().reset_index(name='Total')

In [30]:
all_data.head()

,REPORTED_DATE,OFFENSE_CATEGORY_ID,Total
0,2012-01-31,murder,0
1,2016-05-31,murder,1
2,2013-01-31,arson,1
3,2016-12-31,murder,1
4,2014-12-31,murder,1


In [31]:
all_data['Total'] = all_data['Total'].astype(int)

In [32]:
goal = all_data[all_data['REPORTED_DATE'] == '2017-8-31'] \
 .reset_index(drop=True)

In [33]:
goal['Total_Goal'] = goal['Total'].mul(.8).astype(int)

In [34]:
goal['Total_Goal'].dtype

dtype('int32')

In [35]:
goal.head()

,REPORTED_DATE,OFFENSE_CATEGORY_ID,Total,Total_Goal
0,2017-08-31,murder,7,5
1,2017-08-31,arson,7,5
2,2017-08-31,sexual-assault,57,45
3,2017-08-31,robbery,108,86
4,2017-08-31,white-collar-crime,138,110


In [36]:
pd.merge_asof(goal, all_data, left_on='Total_Goal',
 right_on='Total', by='OFFENSE_CATEGORY_ID',
 suffixes=('_Current', '_Last'))

,REPORTED_DATE_Current,OFFENSE_CATEGORY_ID,Total_Current,Total_Goal,REPORTED_DATE_Last,Total_Last
0,2017-08-31,murder,7,5,2017-05-31,5
1,2017-08-31,arson,7,5,2012-01-31,5
2,2017-08-31,sexual-assault,57,45,2013-01-31,45
3,2017-08-31,robbery,108,86,2015-03-31,86
4,2017-08-31,white-collar-crime,138,110,2016-10-31,110
5,2017-08-31,aggravated-assault,195,156,2016-05-31,154
6,2017-08-31,other-crimes-against-persons,376,300,2014-04-30,285
7,2017-08-31,burglary,432,345,2012-01-31,343
8,2017-08-31,auto-theft,599,479,2017-07-31,477
9,2017-08-31,drug-alcohol,636,508,2015-05-31,505


In [37]:
pd.Period(year=2012, month=5, day=17, hour=14, minute=20, freq='T')

Period('2012-05-17 14:20', 'T')

In [38]:
ad_period = crime_sort.groupby([lambda x: x.to_period('M'),
 'OFFENSE_CATEGORY_ID']).size()


In [39]:
ad_period = ad_period.sort_values() \
 .reset_index(name='Total') \
 .rename(columns={'level_0':'REPORTED_DATE'})

In [40]:
ad_period.head()

,REPORTED_DATE,OFFENSE_CATEGORY_ID,Total
0,2012-01,murder,0
1,2016-05,murder,1
2,2013-01,arson,1
3,2016-12,murder,1
4,2014-12,murder,1


In [41]:
cols = ['OFFENSE_CATEGORY_ID', 'Total']

In [42]:
all_data[cols].equals(ad_period[cols])

False

In [43]:
aug_2018 = pd.Period('2017-8', freq='M')

In [44]:
goal_period = ad_period[ad_period['REPORTED_DATE'] == aug_2018] \
.reset_index(drop=True)


In [45]:
goal_period['Total_Goal'] = goal_period['Total'].mul(.8).astype(int)


In [48]:
ad_period['Total'] = ad_period['Total'].astype(int)

In [49]:
pd.merge_asof(goal_period, ad_period, left_on='Total_Goal', 
              right_on='Total', by='OFFENSE_CATEGORY_ID',
 suffixes=('_Current', '_Last')).head()

,REPORTED_DATE_Current,OFFENSE_CATEGORY_ID,Total_Current,Total_Goal,REPORTED_DATE_Last,Total_Last
0,2017-08,murder,7,5,2017-05,5
1,2017-08,arson,7,5,2012-01,5
2,2017-08,sexual-assault,57,45,2013-01,45
3,2017-08,robbery,108,86,2015-03,86
4,2017-08,white-collar-crime,138,110,2016-10,110
